# 📰 News Retrieval and Embedding System - RPP RSS Feed

## Task 1: News Retrieval and Embedding System

**Objective**: Ingest the latest news from RPP Perú (https://rpp.pe/rss), embed them using SentenceTransformers, and build a retrieval system using ChromaDB orchestrated with LangChain.

### Pipeline Overview:
0. **Load Data** - Extract 50 latest news from RPP RSS feed
1. **Tokenization** - Tokenize with tiktoken and analyze token counts
2. **Embedding** - Generate embeddings with sentence-transformers/all-MiniLM-L6-v2
3. **Create/Upsert Chroma** - Store in ChromaDB with metadata
4. **Query Results** - Semantic similarity search with DataFrame output
5. **LangChain Orchestration** - End-to-end modular pipeline

---

## 📦 Setup and Imports

Import all required libraries and custom modules from the `src/` directory.

In [71]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
from datetime import datetime

# Import custom modules
from rss_loader import load_rss_feed, format_news_for_embedding
from tokenizer import tokenize_text, count_tokens, should_chunk
from embeddings import EmbeddingGenerator
from vector_store import ChromaDBStore
from langchain_pipeline import NewsRetrievalPipeline
from utils import create_results_dataframe, display_results

print("✅ All modules imported successfully!")

✅ All modules imported successfully!


## Step 0️⃣: Load Data from RPP RSS Feed

Load the latest 50 news items from RPP Perú RSS feed.

**Requirements:**
- Use `feedparser` to extract news items
- Each record includes: `title`, `description`, `link`, `published` (date)

In [53]:
# Load RSS feed
print("📡 Loading RSS feed from RPP Perú...")
news_items = load_rss_feed(url="https://rpp.pe/rss", max_items=50)

print(f"✅ Loaded {len(news_items)} news items")
print("\nFirst 3 news items:")
for i, item in enumerate(news_items[:3], 1):
    print(f"\n{i}. {item['title']}")
    print(f"   Published: {item['published']}")
    print(f"   Link: {item['link']}")
    print(f"   Description: {item['description'][:100]}...")

📡 Loading RSS feed from RPP Perú...
✅ Loaded 50 news items

First 3 news items:

1. Revisa el link oficial del retiro AFP 2025 para registrar tu solicitud de desembolso
   Published: Wed, 22 Oct 2025 15:45:08 -0500
   Link: https://rpp.pe/economia/economia/retiro-afp-2025-link-oficial-para-registrar-tu-solicitud-y-pasos-para-solicitar-desembolso-4-uit-de-fondos-noticia-1660126
   Description: La solicitud es gratuita y 100 % digital, mediante los portales oficiales de cada AFP. Evita enlaces...

2. Ate: Policía detiene a un hombre por presuntamente realizar propuestas sexuales a una niña de 11 años
   Published: Wed, 22 Oct 2025 15:39:21 -0500
   Link: https://rpp.pe/peru/la-libertad/ate-policia-detiene-a-un-hombre-por-presuntamente-realizar-propuestas-sexuales-a-una-nina-de-11-anos-noticia-1660509
   Description: El hombre de 29 años fue arrestado en su propia su casa esta madrugada. Según el coronel Johnny Huam...

3. Real Madrid vs. Juventus hoy EN VIVO: ¿a qué hora juegan y dónde v

In [54]:
# Create DataFrame for visualization
df_news = pd.DataFrame(news_items)
print("\n📊 News DataFrame:")
print(df_news.head())
print(f"\nShape: {df_news.shape}")


📊 News DataFrame:
                                               title  \
0  Revisa el link oficial del retiro AFP 2025 par...   
1  Ate: Policía detiene a un hombre por presuntam...   
2  Real Madrid vs. Juventus hoy EN VIVO: ¿a qué h...   
3  Gabinete de Ernesto Álvarez se presenta esta t...   
4  LDU Quito vs. Palmeiras: ¿a qué hora juegan y ...   

                                         description  \
0  La solicitud es gratuita y 100 % digital, medi...   
1  El hombre de 29 años fue arrestado en su propi...   
2  En el Santiago Bernabéu, Real Madrid y Juventu...   
3  El premier Ernesto Álvarez solicitó presentars...   
4  Consulta todos los detalles de lo que será el ...   

                                                link  \
0  https://rpp.pe/economia/economia/retiro-afp-20...   
1  https://rpp.pe/peru/la-libertad/ate-policia-de...   
2  https://rpp.pe/futbol/champions-league/real-ma...   
3  https://rpp.pe/politica/gobierno/gabinete-de-e...   
4  https://rpp.pe/futbol/co

## Step 1️⃣: Tokenization with tiktoken

Tokenize a sample article to understand token counts and determine if chunking is needed.

**Requirements:**
- Use `tiktoken` for tokenization
- Compute `num_tokens` for sample articles
- Decide if chunking is needed based on model context limits (8192 tokens)

In [55]:
# Select a sample article
sample_article = format_news_for_embedding(news_items[0])

print("📝 Sample Article:")
print(sample_article)
print("\n" + "="*80)

📝 Sample Article:
Revisa el link oficial del retiro AFP 2025 para registrar tu solicitud de desembolso. La solicitud es gratuita y 100 % digital, mediante los portales oficiales de cada AFP. Evita enlaces no verificados para prevenir intentos de fraude.



In [56]:
# Tokenize and count tokens
tokens = tokenize_text(sample_article)
num_tokens = count_tokens(sample_article)

print(f"\n🔢 Token Analysis:")
print(f"   Number of tokens: {num_tokens}")
print(f"   First 10 token IDs: {tokens[:10]}")

# Check if chunking is needed
needs_chunking = should_chunk(sample_article, max_tokens=8192)
print(f"\n   Chunking needed (>8192 tokens): {needs_chunking}")


🔢 Token Analysis:
   Number of tokens: 59
   First 10 token IDs: [697, 94633, 658, 2723, 75540, 1624, 2160, 8869, 27746, 220]

   Chunking needed (>8192 tokens): False


In [57]:
# Analyze token counts for all articles
token_counts = [count_tokens(format_news_for_embedding(item)) for item in news_items]

print("\n📊 Token Statistics Across All Articles:")
print(f"   Average tokens: {np.mean(token_counts):.2f}")
print(f"   Min tokens: {np.min(token_counts)}")
print(f"   Max tokens: {np.max(token_counts)}")
print(f"   Median tokens: {np.median(token_counts):.2f}")


📊 Token Statistics Across All Articles:
   Average tokens: 73.80
   Min tokens: 45
   Max tokens: 136
   Median tokens: 72.00


## Step 2️⃣: Generate Embeddings with SentenceTransformers

Use the `sentence-transformers/all-MiniLM-L6-v2` model to generate embeddings.

**Requirements:**
- Model: `sentence-transformers/all-MiniLM-L6-v2`
- Generate 384-dimensional embeddings
- Store embeddings alongside text and metadata

In [58]:
# Initialize embedding generator
print("🤖 Initializing SentenceTransformer model...")
embedding_generator = EmbeddingGenerator(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("✅ Model loaded!")

🤖 Initializing SentenceTransformer model...


/Users/gabrielsaco/anaconda3/envs/rpp/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Model loaded!


In [59]:
# Generate embeddings for all news items
print("\n🔄 Generating embeddings for all news items...")
texts = [format_news_for_embedding(item) for item in news_items]
embeddings = embedding_generator.embed_texts(texts)

print(f"✅ Generated {len(embeddings)} embeddings")
print(f"   Embedding dimension: {embeddings[0].shape[0]}")
print(f"   Sample embedding (first 10 values): {embeddings[0][:10]}")


🔄 Generating embeddings for all news items...
✅ Generated 50 embeddings
   Embedding dimension: 384
   Sample embedding (first 10 values): [-0.12686767  0.00906485  0.01296752 -0.07259791 -0.07967388  0.012302
 -0.00340147  0.01742343  0.02747683  0.04642558]


## Step 3️⃣: Create or Upsert ChromaDB Collection

Store documents, metadata, and embeddings in ChromaDB.

**Requirements:**
- Use ChromaDB to store documents, metadata, and embeddings
- Implement upsert operation for updates
- Support similarity search by keyword or description

In [60]:
# Initialize ChromaDB store
print("💾 Initializing ChromaDB store...")
chroma_store = ChromaDBStore(
    collection_name="rpp_news",
    persist_directory="../chroma_db"
)
print("✅ ChromaDB store initialized!")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


💾 Initializing ChromaDB store...
✅ ChromaDB store initialized!


In [61]:
# Prepare metadata
metadatas = [
    {
        'title': item['title'],
        'description': item['description'],
        'link': item['link'],
        'published': item['published']
    }
    for item in news_items
]

# Generate unique IDs
ids = [f"news_{i}" for i in range(len(news_items))]

print(f"📝 Prepared {len(metadatas)} metadata entries")

📝 Prepared 50 metadata entries


In [62]:
# Upsert documents to ChromaDB
print("\n⬆️  Upserting documents to ChromaDB...")
chroma_store.upsert_documents(
    documents=texts,
    metadatas=metadatas,
    embeddings=embeddings.tolist(),
    ids=ids
)

collection_count = chroma_store.get_collection_count()
print(f"✅ Collection now contains {collection_count} documents")


⬆️  Upserting documents to ChromaDB...
✅ Collection now contains 50 documents


## Step 4️⃣: Query and Retrieve Results

Query the system with "Últimas noticias de economía" and display results in a DataFrame.

**Requirements:**
- Query with prompts like "Últimas noticias de economía"
- Display results in pandas DataFrame
- Columns: `title | description | link | date_published`
- Show top 5-10 most relevant results

In [63]:
# Query the collection
query_text = "Últimas noticias de economía"
print(f"🔍 Querying: '{query_text}'")

results = chroma_store.query(
    query_texts=[query_text],
    n_results=10
)

print(f"✅ Found {len(results['metadatas'][0])} results")

🔍 Querying: 'Últimas noticias de economía'
✅ Found 10 results
✅ Found 10 results


In [64]:
# Create and display results DataFrame
df_results = create_results_dataframe(results)

print("\n📊 Query Results:")
display(df_results)

# Save to CSV
output_path = "../outputs/query_results_economia.csv"
df_results.to_csv(output_path, index=False)
print(f"\n💾 Results saved to: {output_path}")


📊 Query Results:


,title,description,link,date_published
0,Consejo Fiscal pide que el TC revise las más d...,"Alonso Segura alerta sobre la ""avalancha enorm...",https://rpp.pe/economia/economia/consejo-fisca...,"Wed, 22 Oct 2025 10:15:26 -0500"
1,"José Jerí sobre estado de emergencia: ""Las dec...",Durante la ceremonia por el aniversario de la ...,https://rpp.pe/politica/gobierno/jose-jeri-sob...,"Wed, 22 Oct 2025 12:31:52 -0500"
2,Agua Marina se retira temporalmente los escena...,El cantante Lucho Granda señaló que la decisió...,https://rpp.pe/musica/nacional/agua-marina-se-...,"Wed, 22 Oct 2025 11:59:24 -0500"
3,"Congreso propone sueldos de hasta S/ 17,500 en...",El congresista Guido Bellido presentó un proye...,https://rpp.pe/economia/economia/alza-salarial...,"Wed, 22 Oct 2025 13:30:02 -0500"
4,Tragedia en Venezuela: dos personas mueren tra...,Dos personas perdieron la vida este miércoles ...,https://rpp.pe/mundo/venezuela/tragedia-en-ven...,"Wed, 22 Oct 2025 12:36:16 -0500"
5,Álvarez dice que posible ampliación del Reinfo...,El jefe del Gabinete Ministerial indicó que el...,https://rpp.pe/politica/gobierno/alvarez-sobre...,"Wed, 22 Oct 2025 14:08:40 -0500"
6,Intriga en Corinthians: André Carrillo respond...,"De acuerdo con medios de Porto Alegre, André C...",https://rpp.pe/futbol/peruanos-en-el-mundo/and...,"Wed, 22 Oct 2025 13:05:31 -0500"
7,¿Será necesario prorrogar el estado de emergen...,"En Ampliación de Noticias, Rubén Cano consider...",https://rpp.pe/lima/seguridad/estado-de-emerge...,"Wed, 22 Oct 2025 13:34:50 -0500"
8,“Todo está podrido adentro”: exinterno adviert...,"En el programa Encendidos, un expresidiario co...",https://rpp.pe/videos/encendidos/todo-esta-pod...,"Wed, 22 Oct 2025 12:30:27 -0500"
9,Abogada de Susana Villarán pide que se aplique...,"Además, Merly Edquen, abogada de la exalcaldes...",https://rpp.pe/politica/judiciales/abogada-de-...,"Wed, 22 Oct 2025 13:40:26 -0500"



💾 Results saved to: ../outputs/query_results_economia.csv


In [65]:
# Try another query - sports
query_text_sports = "Noticias de deportes y fútbol"
print(f"\n🔍 Querying: '{query_text_sports}'")

results_sports = chroma_store.query(
    query_texts=[query_text_sports],
    n_results=10
)

df_results_sports = create_results_dataframe(results_sports)
print("\n📊 Sports Query Results:")
display(df_results_sports)


🔍 Querying: 'Noticias de deportes y fútbol'

📊 Sports Query Results:

📊 Sports Query Results:


,title,description,link,date_published
0,"'Tunche' Rivera: ""Vamos a salir con todo ante ...",El delantero de Universitario apuntó que busca...,https://rpp.pe/futbol/descentralizado/universi...,"Wed, 22 Oct 2025 14:13:17 -0500"
1,¿Será necesario prorrogar el estado de emergen...,"En Ampliación de Noticias, Rubén Cano consider...",https://rpp.pe/lima/seguridad/estado-de-emerge...,"Wed, 22 Oct 2025 13:34:50 -0500"
2,Xiomy Kanashiro descarta reunirse con Yahaira ...,Luego de negar planes de boda con Jefferson Fa...,https://rpp.pe/famosos/farandula/xiomy-kanashi...,"Wed, 22 Oct 2025 13:38:04 -0500"
3,"Partidos de hoy, miércoles 22 de octubre del 2...",Fútbol EN VIVO | Horarios y canales de TV para...,https://rpp.pe/futbol/futbol-mundial/partidos-...,"Wed, 22 Oct 2025 13:45:34 -0500"
4,Tragedia en Venezuela: dos personas mueren tra...,Dos personas perdieron la vida este miércoles ...,https://rpp.pe/mundo/venezuela/tragedia-en-ven...,"Wed, 22 Oct 2025 12:36:16 -0500"
5,Copa Libertadores: Liga de Quito y Palmeiras s...,Palmeiras se mide ante Liga de Quito en el est...,https://rpp.pe/futbol/ligas-internacionales/co...,"Wed, 22 Oct 2025 14:39:40 -0500"
6,LDU Quito vs. Palmeiras: ¿a qué hora juegan y ...,Consulta todos los detalles de lo que será el ...,https://rpp.pe/futbol/copa-libertadores/ldu-qu...,"Wed, 22 Oct 2025 15:33:45 -0500"
7,Real Madrid vs. Juventus hoy EN VIVO: ¿a qué h...,"En el Santiago Bernabéu, Real Madrid y Juventu...",https://rpp.pe/futbol/champions-league/real-ma...,"Wed, 22 Oct 2025 15:04:46 -0500"
8,"José Jerí sobre estado de emergencia: ""Las dec...",Durante la ceremonia por el aniversario de la ...,https://rpp.pe/politica/gobierno/jose-jeri-sob...,"Wed, 22 Oct 2025 12:31:52 -0500"
9,Ate: Policía detiene a un hombre por presuntam...,El hombre de 29 años fue arrestado en su propi...,https://rpp.pe/peru/la-libertad/ate-policia-de...,"Wed, 22 Oct 2025 15:39:21 -0500"


In [66]:
# Try another query - politics
query_text_politics = "Noticias de política y gobierno"
print(f"\n🔍 Querying: '{query_text_politics}'")

results_politics = chroma_store.query(
    query_texts=[query_text_politics],
    n_results=10
)

df_results_politics = create_results_dataframe(results_politics)
print("\n📊 Politics Query Results:")
display(df_results_politics)


🔍 Querying: 'Noticias de política y gobierno'

📊 Politics Query Results:

📊 Politics Query Results:


,title,description,link,date_published
0,Gabinete de Ernesto Álvarez se presenta esta t...,El premier Ernesto Álvarez solicitó presentars...,https://rpp.pe/politica/gobierno/gabinete-de-e...,"Wed, 22 Oct 2025 15:37:36 -0500"
1,"José Jerí sobre estado de emergencia: ""Las dec...",Durante la ceremonia por el aniversario de la ...,https://rpp.pe/politica/gobierno/jose-jeri-sob...,"Wed, 22 Oct 2025 12:31:52 -0500"
2,TC respalda al JNE y suspende fallos judiciale...,El Poder Judicial había ordenado al Jurado Nac...,https://rpp.pe/politica/elecciones/tc-respalda...,"Wed, 22 Oct 2025 12:08:58 -0500"
3,Álvarez dice que posible ampliación del Reinfo...,El jefe del Gabinete Ministerial indicó que el...,https://rpp.pe/politica/gobierno/alvarez-sobre...,"Wed, 22 Oct 2025 14:08:40 -0500"
4,¿Será necesario prorrogar el estado de emergen...,"En Ampliación de Noticias, Rubén Cano consider...",https://rpp.pe/lima/seguridad/estado-de-emerge...,"Wed, 22 Oct 2025 13:34:50 -0500"
5,Sentencian a 21 años de prisión a sicario vene...,"El asesino pudo ser identificado, en parte, gr...",https://rpp.pe/lima/policiales/comas-sentencia...,"Wed, 22 Oct 2025 14:50:19 -0500"
6,Abogada de Susana Villarán pide que se aplique...,"Además, Merly Edquen, abogada de la exalcaldes...",https://rpp.pe/politica/judiciales/abogada-de-...,"Wed, 22 Oct 2025 13:40:26 -0500"
7,Ate: Policía detiene a un hombre por presuntam...,El hombre de 29 años fue arrestado en su propi...,https://rpp.pe/peru/la-libertad/ate-policia-de...,"Wed, 22 Oct 2025 15:39:21 -0500"
8,Tragedia en Venezuela: dos personas mueren tra...,Dos personas perdieron la vida este miércoles ...,https://rpp.pe/mundo/venezuela/tragedia-en-ven...,"Wed, 22 Oct 2025 12:36:16 -0500"
9,"'Tunche' Rivera: ""Vamos a salir con todo ante ...",El delantero de Universitario apuntó que busca...,https://rpp.pe/futbol/descentralizado/universi...,"Wed, 22 Oct 2025 14:13:17 -0500"


## Step 5️⃣: LangChain Orchestration Pipeline

Implement the complete end-to-end pipeline using LangChain.

**Requirements:**
- End-to-end pipeline: Load RSS → Tokenize → Embed → Store → Retrieve
- Each step should be modular (functions or LangChain chains)
- Demonstrate complete pipeline execution with query results

In [67]:
# Initialize LangChain pipeline
print("🔗 Initializing LangChain Pipeline...")
langchain_pipeline = NewsRetrievalPipeline(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    persist_directory="../chroma_db"
)
print("✅ LangChain pipeline initialized!")

🔗 Initializing LangChain Pipeline...


/Users/gabrielsaco/anaconda3/envs/rpp/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ LangChain pipeline initialized!


In [68]:
# Load fresh RSS feed for LangChain demo
print("\n📡 Loading fresh RSS feed...")
fresh_news = load_rss_feed(url="https://rpp.pe/rss", max_items=50)
print(f"✅ Loaded {len(fresh_news)} fresh news items")


📡 Loading fresh RSS feed...
✅ Loaded 50 fresh news items
✅ Loaded 50 fresh news items


In [69]:
# Step 1: Load and Process with LangChain
print("\n🔄 Step 1: Loading and processing documents...")
documents = langchain_pipeline.load_and_process(fresh_news)
print(f"✅ Created {len(documents)} LangChain documents")
print(f"   Sample document content: {documents[0].page_content[:100]}...")
print(f"   Sample metadata: {documents[0].metadata}")


🔄 Step 1: Loading and processing documents...
✅ Created 50 LangChain documents
   Sample document content: Revisa el link oficial del retiro AFP 2025 para registrar tu solicitud de desembolso. La solicitud e...
   Sample metadata: {'title': 'Revisa el link oficial del retiro AFP 2025 para registrar tu solicitud de desembolso', 'link': 'https://rpp.pe/economia/economia/retiro-afp-2025-link-oficial-para-registrar-tu-solicitud-y-pasos-para-solicitar-desembolso-4-uit-de-fondos-noticia-1660126', 'published': 'Wed, 22 Oct 2025 15:45:08 -0500', 'description': 'La solicitud es gratuita y 100 % digital, mediante los portales oficiales de cada AFP. Evita enlaces no verificados para prevenir intentos de fraude.'}


In [70]:
# Step 2: Create Vector Store
print("\n🔄 Step 2: Creating vector store...")
langchain_pipeline.create_vectorstore(documents)
print("✅ Vector store created!")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



🔄 Step 2: Creating vector store...


OperationalError: attempt to write a readonly database

In [ ]:
# Step 3: Query with LangChain
print("\n🔄 Step 3: Querying vector store...")
query = "Últimas noticias de economía"
df_langchain_results = langchain_pipeline.query(query, k=10)

print(f"\n📊 LangChain Query Results for: '{query}'")
display(df_langchain_results)

# Save results
output_path_lc = "../outputs/langchain_query_results.csv"
df_langchain_results.to_csv(output_path_lc, index=False)
print(f"\n💾 Results saved to: {output_path_lc}")


🔄 Step 3: Querying vector store...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



📊 LangChain Query Results for: 'Últimas noticias de economía'


,title,description,link,date_published
0,Consejo Fiscal pide que el TC revise las más d...,"Alonso Segura alerta sobre la ""avalancha enorm...",https://rpp.pe/economia/economia/consejo-fisca...,"Wed, 22 Oct 2025 10:15:26 -0500"
1,"José Jerí sobre estado de emergencia: ""Las dec...",Durante la ceremonia por el aniversario de la ...,https://rpp.pe/politica/gobierno/jose-jeri-sob...,"Wed, 22 Oct 2025 12:31:52 -0500"
2,Agua Marina se retira temporalmente los escena...,El cantante Lucho Granda señaló que la decisió...,https://rpp.pe/musica/nacional/agua-marina-se-...,"Wed, 22 Oct 2025 11:59:24 -0500"
3,"Congreso propone sueldos de hasta S/ 17,500 en...",El congresista Guido Bellido presentó un proye...,https://rpp.pe/economia/economia/alza-salarial...,"Wed, 22 Oct 2025 13:30:02 -0500"
4,Tragedia en Venezuela: dos personas mueren tra...,Dos personas perdieron la vida este miércoles ...,https://rpp.pe/mundo/venezuela/tragedia-en-ven...,"Wed, 22 Oct 2025 12:36:16 -0500"
5,Álvarez dice que posible ampliación del Reinfo...,El jefe del Gabinete Ministerial indicó que el...,https://rpp.pe/politica/gobierno/alvarez-sobre...,"Wed, 22 Oct 2025 14:08:40 -0500"
6,Intriga en Corinthians: André Carrillo respond...,"De acuerdo con medios de Porto Alegre, André C...",https://rpp.pe/futbol/peruanos-en-el-mundo/and...,"Wed, 22 Oct 2025 13:05:31 -0500"
7,¿Será necesario prorrogar el estado de emergen...,"En Ampliación de Noticias, Rubén Cano consider...",https://rpp.pe/lima/seguridad/estado-de-emerge...,"Wed, 22 Oct 2025 13:34:50 -0500"
8,“Todo está podrido adentro”: exinterno adviert...,"En el programa Encendidos, un expresidiario co...",https://rpp.pe/videos/encendidos/todo-esta-pod...,"Wed, 22 Oct 2025 12:30:27 -0500"
9,Abogada de Susana Villarán pide que se aplique...,"Además, Merly Edquen, abogada de la exalcaldes...",https://rpp.pe/politica/judiciales/abogada-de-...,"Wed, 22 Oct 2025 13:40:26 -0500"



💾 Results saved to: ../outputs/langchain_query_results.csv


In [ ]:
# Complete Pipeline Demo
print("\n🚀 Running Complete End-to-End Pipeline...\n")

# Create a new pipeline instance
complete_pipeline = NewsRetrievalPipeline(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    persist_directory="../chroma_db"
)

# Run complete pipeline
query_complete = "Noticias sobre tecnología e innovación"
df_complete = complete_pipeline.run_pipeline(
    news_items=fresh_news,
    query_text=query_complete,
    k=10
)

print(f"\n📊 Complete Pipeline Results for: '{query_complete}'")
display(df_complete)

print("\n✅ Complete pipeline executed successfully!")


🚀 Running Complete End-to-End Pipeline...



/Users/gabrielsaco/anaconda3/envs/rpp/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



📊 Complete Pipeline Results for: 'Noticias sobre tecnología e innovación'


,title,description,link,date_published
0,Agua Marina se retira temporalmente los escena...,El cantante Lucho Granda señaló que la decisió...,https://rpp.pe/musica/nacional/agua-marina-se-...,"Wed, 22 Oct 2025 11:59:24 -0500"
1,Agua Marina se retira temporalmente los escena...,El cantante Lucho Granda señaló que la decisió...,https://rpp.pe/musica/nacional/agua-marina-se-...,"Wed, 22 Oct 2025 11:59:24 -0500"
2,"José Jerí sobre estado de emergencia: ""Las dec...",Durante la ceremonia por el aniversario de la ...,https://rpp.pe/politica/gobierno/jose-jeri-sob...,"Wed, 22 Oct 2025 12:31:52 -0500"
3,"José Jerí sobre estado de emergencia: ""Las dec...",Durante la ceremonia por el aniversario de la ...,https://rpp.pe/politica/gobierno/jose-jeri-sob...,"Wed, 22 Oct 2025 12:31:52 -0500"
4,Intriga en Corinthians: André Carrillo respond...,"De acuerdo con medios de Porto Alegre, André C...",https://rpp.pe/futbol/peruanos-en-el-mundo/and...,"Wed, 22 Oct 2025 13:05:31 -0500"
5,Intriga en Corinthians: André Carrillo respond...,"De acuerdo con medios de Porto Alegre, André C...",https://rpp.pe/futbol/peruanos-en-el-mundo/and...,"Wed, 22 Oct 2025 13:05:31 -0500"
6,¿Será necesario prorrogar el estado de emergen...,"En Ampliación de Noticias, Rubén Cano consider...",https://rpp.pe/lima/seguridad/estado-de-emerge...,"Wed, 22 Oct 2025 13:34:50 -0500"
7,¿Será necesario prorrogar el estado de emergen...,"En Ampliación de Noticias, Rubén Cano consider...",https://rpp.pe/lima/seguridad/estado-de-emerge...,"Wed, 22 Oct 2025 13:34:50 -0500"
8,Sentencian a 21 años de prisión a sicario vene...,"El asesino pudo ser identificado, en parte, gr...",https://rpp.pe/lima/policiales/comas-sentencia...,"Wed, 22 Oct 2025 14:50:19 -0500"
9,Sentencian a 21 años de prisión a sicario vene...,"El asesino pudo ser identificado, en parte, gr...",https://rpp.pe/lima/policiales/comas-sentencia...,"Wed, 22 Oct 2025 14:50:19 -0500"



✅ Complete pipeline executed successfully!


---

## 📊 Summary: 5 Most Recent Retrieved Items

Display the 5 most recent items retrieved from the economic news query.

In [ ]:
# Display the 5 most recent retrieved items from the economic query
print("=" * 100)
print("📊 TOP 5 MOST RELEVANT ECONOMIC NEWS ARTICLES")
print("=" * 100)

df_top5 = df_results.head(5)
for idx, row in df_top5.iterrows():
    print(f"\n{idx + 1}. {row['title']}")
    print(f"   📅 Published: {row['date_published']}")
    print(f"   📝 Description: {row['description'][:150]}...")
    print(f"   🔗 Link: {row['link']}")
    print("-" * 100)

print("\n✅ Task 1 Complete! All requirements satisfied.")

📊 TOP 5 MOST RELEVANT ECONOMIC NEWS ARTICLES

1. Consejo Fiscal pide que el TC revise las más de 200 leyes aprobadas por el Congreso que comprometen la estabilidad fiscal
   📅 Published: Wed, 22 Oct 2025 10:15:26 -0500
   📝 Description: Alonso Segura alerta sobre la "avalancha enorme" de gasto, cifrada en S/ 36,000 millones, y advierte que el país podría perder el grado de inversión s...
   🔗 Link: https://rpp.pe/economia/economia/consejo-fiscal-pide-que-el-tc-revise-las-mas-de-200-leyes-aprobadas-por-el-congreso-que-comprometen-la-estabilidad-fiscal-noticia-1660497
----------------------------------------------------------------------------------------------------

2. José Jerí sobre estado de emergencia: "Las decisiones se toman pensando en el bien de la mayoría, aunque a algunos no les guste"
   📅 Published: Wed, 22 Oct 2025 12:31:52 -0500
   📝 Description: Durante la ceremonia por el aniversario de la Dirección de Seguridad del Estado, el presidente José Jerí respaldó el accionar p